## 1. 데이터 준비

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt




### 1.1 이미지

In [2]:
from PIL import Image
import os, glob

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"


### 1.2 data set 생성

#### 1.2.1 train set

In [3]:
def load_data(img_path):
    number_of_data=9300
    img_size=28
    color=3
    imgs=np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)
                  
    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=0    # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=2   # 보 : 2
        idx=idx+1

    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")                  
    return imgs, labels   
                  
             
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0
                  
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 9300 입니다.
x_train shape: (9300, 28, 28, 3)
y_train shape: (9300,)


#### 오류1
File "<ipython-input-6-52fa99673387>", line 8
    idx=0
      ^
SyntaxError: invalid syntax
    
 
    -->dtype=np.int32).reshape(number_of_data,img_size,img_size,color) 구문누락


#### 오류2
학습데이터 이미지 개수 표기오류


-----> 이미지 파일 확장자 오류. jpeg-->jpg로 변경


#### 1.2.2 test set

In [4]:
def load_data(img_path):
    number_of_data=300
    img_size=28
    color=3
    imgs=np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)
                  
    idx=0
    for file in glob.iglob(img_path+'/scissor_test/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=0    # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock_test/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper_test/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img
        labels[idx]=2   # 보 : 2
        idx=idx+1

    print("test데이터(x_test)의 이미지 개수는",idx,"입니다.")                  
    return imgs, labels   
                  
             
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0
                  
print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))


test데이터(x_test)의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


## 2. 네트워크 설계

### 2.1 Sequenential Model 사용

In [5]:
n_channel_1=32
n_channel_2=64
n_dense=128

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))


## 3. 네트워크 학습

In [6]:
n_train_epoch=10

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=n_train_epoch)

Epoch 1/10
291/291 [==============================] - 1s 4ms/step - loss: 7.8872 - accuracy: 0.4966
Epoch 2/10
291/291 [==============================] - 1s 4ms/step - loss: 0.4603 - accuracy: 0.8132
Epoch 3/10
291/291 [==============================] - 1s 4ms/step - loss: 0.2165 - accuracy: 0.9218
Epoch 4/10
291/291 [==============================] - 1s 4ms/step - loss: 0.1346 - accuracy: 0.9533
Epoch 5/10
291/291 [==============================] - 1s 4ms/step - loss: 0.0886 - accuracy: 0.9713
Epoch 6/10
291/291 [==============================] - 1s 4ms/step - loss: 0.0590 - accuracy: 0.9809
Epoch 7/10
291/291 [==============================] - 1s 4ms/step - loss: 0.0652 - accuracy: 0.9773
Epoch 8/10
291/291 [==============================] - 1s 4ms/step - loss: 0.0607 - accuracy: 0.9819
Epoch 9/10
291/291 [==============================] - 1s 4ms/step - loss: 0.0520 - accuracy: 0.9829
Epoch 10/10
291/291 [==============================] - 1s 4ms/step - loss: 0.0343 - accuracy: 0.9891

## 4. 네트워크 테스트

In [7]:
test_loss, test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 1.3565 - accuracy: 0.6067
test_loss: 1.35645592212677 
test_accuracy: 0.6066666841506958


#### 결과 1
n_channel_1=16
n_channel_2=32
n_dense=32
n_train_epoch=10

10/10 - 0s - loss: 1.6963 - accuracy: 0.4600
test_loss: 1.6963428258895874 
test_accuracy: 0.46000000834465027------------->60% 미달성



#### 결과 2
n_channel_1=16
n_channel_2=32
n_dense=64
n_train_epoch=10

10/10 - 0s - loss: 1.0978 - accuracy: 0.6000
test_loss: 1.0977530479431152 
test_accuracy: 0.6000000238418579 ------------->60% 달성




#### 결과 3
n_channel_1=16
n_channel_2=64
n_dense=64
n_train_epoch=10

10/10 - 0s - loss: 1.4263 - accuracy: 0.5033
test_loss: 1.4263067245483398 
test_accuracy: 0.503333330154419 ------------->하락




#### 결과 4
n_channel_1=32
n_channel_2=64
n_dense=64
n_train_epoch=10

10/10 - 0s - loss: 1.6690 - accuracy: 0.4967
test_loss: 1.6690047979354858 
test_accuracy: 0.49666666984558105 ------------->하락




#### 결과 5
n_channel_1=32
n_channel_2=64
n_dense=128
n_train_epoch=10

10/10 - 0s - loss: 1.0407 - accuracy: 0.6433
test_loss: 1.0407438278198242 
test_accuracy: 0.6433333158493042 ------------->상승

### dense가 큰 영향을 주는것으로 보임




# 느낀점

처음에 노드 학습을 진행할때만 해도 각 코드가 의미하는바를 하나도 모른채로 그저 복사 붙여넣기에 몇가지 경로만 수정하는식으로 노드를 수행했었다.
다른 노드를 학습하면서 코드가 하는 역할을 조금씩 알게되었고 해당 과제물을 작성할때는 최대한 중복되는 코드를 제거하면서 해보려고 노력했다.

테스트 결과에서 이미지 특징값(Conv2D)은 정확도 향상에 큰 영향을 주지 못했고 dense가 영향을 많이주는것으로 확인했다.

학습을 진행하면서 데이터셋을 분류하여 데이터 총 량을 늘려보고 테스트 결과를 확인하는 과정이 재미있었고 학습도중 다른분들과 의견 교환하는것도 흥미로웠다.
